##Content Based Filtering : Debashish Malakar

In [0]:
import numpy as np
import pandas as pd

In [0]:
df_posts = pd.read_csv("posts.csv")
df_users = pd.read_csv("users.csv")
df_views = pd.read_csv("views.csv")

In [665]:
post_id_renamed = df_posts.rename(index = str,columns = {'_id' : 'post_id'})
df_views_merge = df_views.merge(post_id_renamed,on = 'post_id')
df_views_merge['category'] = df_views_merge['category'].fillna('')
user_id_renamed = df_users.rename(index = str,columns = {'_id' : 'user_id'})
df_views_merge = df_views_merge.merge(user_id_renamed,on = 'user_id')
df_views_merge_user = df_views_merge.drop(columns=['post_id','timestamp','title','category',' post_type'])
df_views_merge = df_views_merge.drop(columns = ['name','timestamp'])
df_views_merge['Content'] = df_views_merge[['title', 'category','gender','academics',' post_type']].apply(lambda x: ' '.join(x), axis = 1)
df_views_merge.head()

,user_id,post_id,title,category,post_type,gender,academics,Content
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork,female,undergraduate,Save Earth. Visual Arts|Graphic Design|Artisti...
1,5df49b32cc709107827fb3c7,5ec1fd0974f7660d73aa0fd5,Daaku,Drawings,artwork,female,undergraduate,Daaku Drawings female undergraduate artwork
2,5df49b32cc709107827fb3c7,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog,female,undergraduate,"Machine Learning”&“Operations"" (MlOps) Compute..."
3,5df49b32cc709107827fb3c7,5ecb72c0eaff6b0c3a58a48e,EID MUBARAK,Photography|Architecture|Visual Arts|Graphic D...,artwork,female,undergraduate,EID MUBARAK Photography|Architecture|Visual Ar...
4,5df49b32cc709107827fb3c7,5ea5aacd10426255a7aa9b71,Photography Composition,,project,female,undergraduate,Photography Composition female undergraduate ...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')



In [0]:
tfv_matrix = tfv.fit_transform(df_views_merge['Content'])

In [0]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [0]:
indices = pd.Series(df_views_merge.index, index=df_views_merge['post_id']).drop_duplicates(keep="first",inplace = False)

In [0]:
def post_index(idxx,sig):
  
  sig_scores = list(enumerate(sig[idxx]))
  score_series = sorted(sig_scores, key=lambda x: x[1], reverse=True)
  sig_scores = score_series[1:]
  recommend_post = [i[0] for i in sig_scores]
  return recommend_post
#post_index(idx,sig)

In [0]:
def recommend(df_views_merge,idxx,indices,post_indices):
  a = np.array(df_views_merge['post_id'].iloc[post_indices])
  original = np.array(df_views_merge['post_id'].iloc[idxx])
  b= []
  for i in a:
    if len(b)>=5:
      break
    if i in b:
      continue
    if i == original:
      continue
    else:
      b.append(i)
    
  ls = []
  for i in range(len(b)):
    if (indices[b[i]]).shape== ():
      ls.append(indices[b[i]])
    else:
      ls.append(indices[b[i]][0])

  print('recommended for {0}'.format(df_views_merge[['post_id','title','category',' post_type']].iloc[idxx]))
  return df_views_merge[['post_id','title','category',' post_type']].iloc[ls]


In [0]:
def id(idx,indices):
  idx = indices[post_id]
  if idx.shape==():
      idxx = idx
  else:
      idxx = idx[0]
  return idxx

In [0]:
indices2 = pd.Series(df_views_merge.index, index=df_views_merge['user_id']).drop_duplicates(keep="first",inplace = False)

In [0]:
def post_index2(idxx2,sig):
  
  sig_scores = list(enumerate(sig[idxx2]))
  score_series = sorted(sig_scores, key=lambda x: x[1], reverse=True)
  sig_scores = score_series[1:]
  recommend_post2 = [i[0] for i in sig_scores]
  return recommend_post2




In [0]:
def recommend2(df_views_merge,idxx2,indices2,post_indices2):
  a = np.array(df_views_merge['user_id'].iloc[post_indices2])
  original = np.array(df_views_merge['user_id'].iloc[idxx2])
  b= []
  for i in a:
    if len(b)>=5:
      break
    if i in b:
      continue
    if i == original:
      continue
    else:
      b.append(i)
    
  ls = []
  for i in range(len(b)):
    if (indices2[b[i]]).shape== ():
      ls.append(indices2[b[i]])
    else:
      ls.append(indices2[b[i]][0])

  
  return df_views_merge[['user_id']].iloc[ls]


In [0]:
def user_post_id(df,users_list,val):
  post_ = []
  for i in users_list:
    post_list = []
    post = df.loc[df['user_id'] == i[0]]
    post2 = post['post_id']
    a = np.array(post2)
    post_.append(a)
  user = df.loc[df['user_id'] ==val]
  user_post = user['post_id']
  b = np.array(user_post)
  c = []
  for j in post_:
    for i in j:
      if i in b:
        continue
      else:
        c.append(i)
  c = c[0:5]
  return c

##User-based Recommendation

In [677]:
user_id = input("Enter user_id :")
idx2 = indices2[user_id]
if idx2.shape==():
    idxx2 = idx2
else:
    idxx2 = idx2[0]

users_list = np.array(recommend2(df_views_merge,idxx2,indices2,post_index2(idxx2,sig))) #uses idxx
orig_user = df_views_merge[['user_id']].iloc[idxx2]
val = orig_user['user_id']
c = user_post_id(df_views,users_list,val)
final_posts=[]
for i in c:
  final_posts.append(indices[i][0])
print('Recommendations for {0}'.format(df_views_merge_user.iloc[idxx2]))
df_views_merge[['post_id','title','category',' post_type']].iloc[final_posts]

Enter user_id :5d9a2982979d5962253c2f81
Recommendations for user_id      5d9a2982979d5962253c2f81
name                  Nischaya Sharma
gender                           male
academics               undergraduate
Name: 1374, dtype: object


,post_id,title,category,post_type
418,5ed3820f76027d35905ccac8,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog
73,5ed1ff0276027d35905cc60d,AWS services and how to launch OS on AWS Cloud,Computer Technology|Cloud Computing,blog
76,5ecf96e876027d35905cbf46,GAN's Part(2),Computer Technology|Machine Learning,blog
579,5ecfa0ca76027d35905cbf57,Recommend Systems Machine Learning,Computer Technology|Machine Learning,blog
89,5ed0e20776027d35905cc2fe,Here are 7 of my favourite books that I've rea...,Literature|Stories|Fictions|Movements in Liter...,blog


##Post-based Recommendation 

In [678]:
#post-based recommendation
post_id = input("Enter post_id :")
idx = indices[post_id]
idxx = id(idx,indices)
recommend(df_views_merge,idxx,indices,post_index(idxx,sig))

Enter post_id :5ed1ff0276027d35905cc60d
recommended for post_id                             5ed1ff0276027d35905cc60d
title         AWS services and how to launch OS on AWS Cloud
category                 Computer Technology|Cloud Computing
 post_type                                              blog
Name: 73, dtype: object


,post_id,title,category,post_type
187,5ed09ad076027d35905cc15f,Understanding Cloud Computing(AWS),Computer Technology|Cloud Computing,blog
178,5e77738bcfc8b713f5ac7b36,Cloud Computing,Computer Technology|Cloud Computing,blog
834,5ed229d976027d35905cc666,Cloud Computing,Computer Technology|Computation|Computer Appli...,blog
1335,5e966ad3a3258347b42f2aaf,8 Best Machine Learning Courses for 2020,Computer Technology|Cloud Computing|Artificial...,blog
1307,5e2f1811c85ab714a7da670d,Abacus,Mathematics|Linear Algebra,blog
